<div style='background-color:orange'>
<a id="TableOfContents"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            TABLE OF CONTENTS:
        </i></b></h1>
    <li><a href='#imports'>Imports</a>
    <li><a href="#acquire">Acquire</a>
    <li><a href='#prepare'>Prepare</a>
    <li><a href="#wrangle">Wrangle</a>
    <li><a href='#misc'>Miscellaneous</a>
    </li>
</div>

<div style='background-color:orange'>
<a id="imports"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Imports
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [1]:
# Vectorization and tables
import numpy as np
import pandas as pd

# Regex
import re

# .py files
import wrangle as w
import acquire as a

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

<div style='background-color:orange'>
<a id="acquire"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Acquire
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    <li><a href='#acquiregabe'>Initial Filter</a>
    <li><a href='#acquiresecond'>Second Filter</a>
    <li><a href='#acquirethird'>Third Filter</a>
    </li>
</div>

In [3]:
master = a.prepare_third_filtered_dataset_version()
master

,crash_id,person_age,person_ethnicity,person_gender,has_motocycle_endorsment,person_injury_severity,vehicle_body_style,vehicle_color,vehicle_make,vehicle_model,vehicle_model_year,vehicle_make_country,injury_binary
0,16189632,37,w - white,1 - male,0,a - suspected serious injury,mc - motorcycle,blu - blue,harley-davidson,fld,2007,USA,1
1,16203470,30,h - hispanic,1 - male,0,b - suspected minor injury,mc - motorcycle,gry - gray,suzuki,gsx-r600,2004,Japan,1
2,16192023,21,w - white,1 - male,0,a - suspected serious injury,mc - motorcycle,blu - blue,yamaha,yzfr6,2017,Japan,1
3,16196720,18,h - hispanic,1 - male,0,b - suspected minor injury,mc - motorcycle,blu - blue,yamaha,rz500,2002,Japan,1
4,16189103,28,w - white,1 - male,1,b - suspected minor injury,mc - motorcycle,blk - black,harley-davidson,fxdf,2009,USA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,19321499,49,w - white,2 - female,1,b - suspected minor injury,mc - motorcycle,blk - black,yamaha,xvs1100,2013,Japan,1
14130,19323296,33,w - white,1 - male,1,a - suspected serious injury,mc - motorcycle,grn - green,kawasaki,kx250,2019,Japan,1
14131,19327850,35,w - white,1 - male,1,a - suspected serious injury,mc - motorcycle,blk - black,honda,cr250,2016,Japan,1
14132,19330330,42,b - black,2 - female,1,b - suspected minor injury,mc - motorcycle,mul - multicolored,honda,cbr650,2016,Japan,1


<div style='background-color:orange'>
<a id="prepare"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Prepare
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

<div style='background-color:orange'>
<a id="misc"></a>
    <h1 style='text-align:center ; top-padding:5px'>
        <b><i>
            Miscellaneous
        </i></b></h1>
    <li><a href='#TableOfContents'>Table of Contents</a>
    </li>
</div>

In [5]:
svcs = pd.read_csv('svcs.csv')

In [11]:
master_dt = master.merge(svcs[['crash_datetime']], how='inner', left_on=master.crash_id, right_on=svcs.crash_id)
master_dt.drop(columns='key_0', inplace=True)
master_dt.crash_datetime = pd.to_datetime(master_dt.crash_datetime)
master_dt.sample()

,crash_id,person_age,person_ethnicity,person_gender,has_motocycle_endorsment,person_injury_severity,vehicle_body_style,vehicle_color,vehicle_make,vehicle_model,vehicle_model_year,vehicle_make_country,injury_binary,crash_datetime
425,16469251,22,h - hispanic,1 - male,0,a - suspected serious injury,mc - motorcycle,blu - blue,kawasaki,kaf300,2007,Japan,1,2018-06-03 05:58:00


In [17]:
pd.DataFrame(master_dt.groupby(master_dt.crash_datetime.dt.year)['person_injury_severity'].value_counts())

count
crash_datetime person_injury_severity             
2018           b - suspected minor injury     1643
               a - suspected serious injury    734
               n - not injured                 232
               k - fatal injury                139
2019           b - suspected minor injury     1570
               a - suspected serious injury    707
               n - not injured                 222
               k - fatal injury                150
2020           b - suspected minor injury     1686
               a - suspected serious injury    736
               n - not injured                 245
               k - fatal injury                185
2021           b - suspected minor injury     1597
               a - suspected serious injury    907
               n - not injured                 219
               k - fatal injury                184
2022           b - suspected minor injury     1629
               a - suspected serious injury    922
               n - not injured                 237
               k - fatal injury                190

In [18]:
injury_dict = {
    'Injury Severity Then Motorcycle Sales' : ['Not Injured', 'Minor Injury', 'Major Injury', 'Fatal', ],
    '2018' : [232, 1643, 734, 139],
    '2019' : [222, 1570, 707, 150],
    '2020' : [245, 1686, 736, 185],
    '2021' : [219, 1597, 907, 184],
    '2022' : [237, 1629, 922, 190]
}

In [20]:
injuries = pd.DataFrame(injury_dict)
injuries.to_csv('injuries.csv', index=False)

In [23]:
moto_sales = {
    'Motorcycle Sales' : ['Motorcycle Sales'],
    '2010' : [439680],
    '2011' : [440900],
    '2012' : [459300],
    '2013' : [465780],
    '2014' : [483530],
    '2015' : [500700],
    '2016' : [487140],
    '2017' : [469730],
    '2018' : [457200],
    '2019' : [467780],
    '2020' : [780000],
}
moto_df = pd.DataFrame(moto_sales)
moto_df.to_csv('moto_sales.csv', index=False)